### Initial imports

In [72]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pprint import pprint  # To tidy up
from bs4 import BeautifulSoup
import bs4
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode
options.add_argument("--no-sandbox")  # Bypass OS security model
options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

In [142]:
# driver.quit()
driver = webdriver.Chrome(options=options)
main_url = "https://www.inmotionhosting.com/"
driver.get(main_url)

wait = WebDriverWait(driver, 10)

try:
    wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "h2.hero-title.color-neutral-color"))
    )
    print("Page loaded successfully")
except Exception as e:
    print("Error loading page:", e)
    driver.quit()
    exit()

Page loaded successfully


In [284]:
if driver.current_url != main_url:
    driver.get(main_url)
    wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "h2.hero-title.color-neutral-color"))
    )
driver.page_source
soup_main = BeautifulSoup(driver.page_source, 'html.parser')

### Section for a debugging

In [285]:
plan_link = [link.get_attribute('href') for link in driver.find_elements(By.LINK_TEXT, "Compare Plans")]


In [286]:
driver.get(plan_link[0])
driver.current_url


'https://www.inmotionhosting.com/shared-hosting'

In [287]:
# btn_1_year = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'imh-term-selector') and text()='6 Month']")))
period_string = '1 Year'
period_btn = driver.find_element(By.XPATH, f"//button[contains(@class, 'imh-term-selector') and text()='{period_string}']")
print(period_btn.text)
driver.execute_script("arguments[0].click();", period_btn)

1 Year


In [288]:
wait.until(EC.text_to_be_present_in_element(
    (By.CSS_SELECTOR, "div.imh-term-selector > button.imh-term-selector.active"),
    period_string
))

True

In [289]:
driver.find_element(By.CSS_SELECTOR, "button.imh-term-selector.active").text

'1 Year'

In [290]:
plans_container_all = BeautifulSoup(driver.page_source, 'html.parser')
plans_container_all.find('div', class_='imh-rostrum-container')

<div class="imh-rostrum-container"><div class="imh-rostrum-card"><h3 class="h3 imh-rostrum-card-title">Core</h3><div class="imh-rostrum-sub-title">Suitable for simple sites, with everything you need included.</div><div class="imh-pricing-container connected-switcher"><div class="imh-switcher"><div class="imh-rostrum-save-ribbon">You Save <span class="dnt">68%</span></div><span class="imh-starting-at-text">Starting at</span><div class="imh-rostrum-starting-at-price-discounted"><span class="rostrum-price">$3.19</span><span class="rostrum-per-price"><small>/mo</small></span></div><div class="imh-rostrum-starting-at-price-normal" style="margin-top: 5px;">Renews at <span class="rostrum-price">$9.99</span><span class="rostrum-per-price">/mo</span></div><button class="atc btn-primary" data-action="entered-cart" data-button='{"campaignhandle": "standard_all_products","productid": "11494","packageid": "847","irid":"","affiliate":"","sirid": "", "opc": ""}' data-click="primary-cta" data-location

In [291]:
plans = plans_container_all.find_all('div', class_='imh-rostrum-card')
plan = plans[0]
plan


<div class="imh-rostrum-card"><h3 class="h3 imh-rostrum-card-title">Core</h3><div class="imh-rostrum-sub-title">Suitable for simple sites, with everything you need included.</div><div class="imh-pricing-container connected-switcher"><div class="imh-switcher"><div class="imh-rostrum-save-ribbon">You Save <span class="dnt">68%</span></div><span class="imh-starting-at-text">Starting at</span><div class="imh-rostrum-starting-at-price-discounted"><span class="rostrum-price">$3.19</span><span class="rostrum-per-price"><small>/mo</small></span></div><div class="imh-rostrum-starting-at-price-normal" style="margin-top: 5px;">Renews at <span class="rostrum-price">$9.99</span><span class="rostrum-per-price">/mo</span></div><button class="atc btn-primary" data-action="entered-cart" data-button='{"campaignhandle": "standard_all_products","productid": "11494","packageid": "847","irid":"","affiliate":"","sirid": "", "opc": ""}' data-click="primary-cta" data-location="/shared-hosting" data-presentatio

In [292]:
plans

[<div class="imh-rostrum-card"><h3 class="h3 imh-rostrum-card-title">Core</h3><div class="imh-rostrum-sub-title">Suitable for simple sites, with everything you need included.</div><div class="imh-pricing-container connected-switcher"><div class="imh-switcher"><div class="imh-rostrum-save-ribbon">You Save <span class="dnt">68%</span></div><span class="imh-starting-at-text">Starting at</span><div class="imh-rostrum-starting-at-price-discounted"><span class="rostrum-price">$3.19</span><span class="rostrum-per-price"><small>/mo</small></span></div><div class="imh-rostrum-starting-at-price-normal" style="margin-top: 5px;">Renews at <span class="rostrum-price">$9.99</span><span class="rostrum-per-price">/mo</span></div><button class="atc btn-primary" data-action="entered-cart" data-button='{"campaignhandle": "standard_all_products","productid": "11494","packageid": "847","irid":"","affiliate":"","sirid": "", "opc": ""}' data-click="primary-cta" data-location="/shared-hosting" data-presentati

In [295]:

title = plan.find('h3', class_='imh-rostrum-card-title').get_text(strip=True)
description = plan.find('div', class_='imh-rostrum-sub-title').get_text(strip=True)

pricing_container = plan.find('div', class_='imh-pricing-container')
if 'switcher' in ' '.join(pricing_container['class']):
    pricing_container = pricing_container.find('div', class_='active')

pricing_container

(
    len(pricing_container),
    title,
    description
)

(5, 'Core', 'Suitable for simple sites, with everything you need included.')

In [296]:

discounted_price = pricing_container.find('div', class_='imh-rostrum-starting-at-price-discounted').find('span', class_='rostrum-price').get_text(strip=True)
normal_price = pricing_container.find('div', class_='imh-rostrum-starting-at-price-normal').find('span', class_='rostrum-price').get_text(strip=True)

discount = pricing_container.find('div', class_='imh-rostrum-save-ribbon').find('span', class_='dnt').get_text(strip=True)

discounted_price, normal_price, discount


('$2.99', '$10.99', '73%')

In [297]:

feature_container = plan.find('ul', class_='imh-rostrum-details-list')
features = feature_container.find_all('li', class_='fa fa-imh-arrow-right tooltips-rostrum', recursive=False)
len(features)

11

In [298]:
# If just use get_text and strip=True, then text will sometimes be concatenated without whitespace.
# It happens when part of a text is inside a tag and other is not.
# So this function gets all text recursively and separates into a list
def get_all_children_text(soup):
    text = []
    for child in soup.children:
        if isinstance(child, str):
            child_text = child.get_text(strip=True)
            if child_text != '':
                text.append(child_text)
        elif isinstance(child, bs4.element.Tag):
            text.extend(get_all_children_text(child))
    return text

get_all_children_text(features[1])


['2', 'Websites']

In [139]:
feature_data = []

for feature in features:
    feature_detail = feature.find(class_='imh-rostrum-detail')
    if 'switcher' in ' '.join(feature_detail['class']):
        feature_detail = feature_detail.find(class_='active')
    if feature_detail.find(class_='imh-rostrum-detail'):
        feature_detail = feature_detail.find(class_='imh-rostrum-detail')
    # print('-------------------- New feature')
    # print(feature_detail)
    all_features = get_all_children_text(feature_detail)
    feature_data.append(' '.join(all_features).replace('\xa0', ''))
    
pprint(feature_data)


['Free domain & SSL',
 '2 Websites',
 '100GB SSD',
 'Unlimited Bandwidth',
 '2x Speed & Performance',
 '10 Email Addresses',
 'Security Suite',
 'Marketing Tools',
 'Hosting Plus Python, Node.JS,  Ruby and GIT version control.',
 'Choice of Data Center Location Includes US and EU Locations',
 'Live Chat Support from Helpful Humans']


In [277]:
def check_none(element):
    return element.get_text(strip=True) if element else ''

def get_plan_features(plan):
    feature_container = plan.find('ul', class_='imh-rostrum-details-list')
    features = feature_container.find_all('li', class_='fa fa-imh-arrow-right tooltips-rostrum', recursive=False)
    
    feature_data = []
    for feature in features:
        feature_detail = feature.find(class_='imh-rostrum-detail')
        if 'switcher' in ' '.join(feature_detail['class']):
            feature_detail = feature_detail.find(class_='active')
            
        if feature_detail.find(class_='imh-rostrum-detail'):
            feature_detail = feature_detail.find(class_='imh-rostrum-detail')
            
        all_features = get_all_children_text(feature_detail)
        feature_data.append(' '.join(all_features).replace('\xa0', ''))
    
    return feature_data

def get_plan_data(plan):
    title = plan.find('h3', class_='imh-rostrum-card-title').get_text(strip=True)
    description = check_none(plan.find('div', class_='imh-rostrum-sub-title'))
    print('plan-title', title)
    pricing_container = plan.find('div', class_='imh-pricing-container')
    if 'switcher' in ' '.join(pricing_container['class']):
        pricing_container = pricing_container.find('div', class_='active')

    discounted_price = check_none(pricing_container.find('div', class_='imh-rostrum-starting-at-price-discounted').find('span', class_='rostrum-price'))
    normal_price = check_none(pricing_container.find('div', class_='imh-rostrum-starting-at-price-normal').find('span', class_='rostrum-price'))
    
    # discount = check_none(pricing_container.find('div', class_='imh-rostrum-save-ribbon').find('span', class_='dnt'))
    # Sometimes discount is not specified
    discount = check_none(pricing_container.select_one('div.imh-rostrum-save-ribbon > span.dnt'))

    features = get_plan_features(plan)
    
    return title, description, discounted_price, normal_price, discount, features


### Web site scraping script

In [282]:
main_url = "https://www.inmotionhosting.com/"

driver.get(main_url)
print('main url', driver.current_url)
period_string = '1 Year'
print('period', period_string)

wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div.imh-rostrum-container"))
)

service_cards = driver.find_element(By.ID, "home-rostrum-2").find_elements(By.CLASS_NAME,
                                                                           "imh-rostrum-card.imh-clickable-card")

# So the elements won't become stale
card_info = list(zip(
    map(lambda x: x.find_element(By.CLASS_NAME, "h3.imh-rostrum-card-title").text, service_cards),
    map(lambda x: x.find_element(By.LINK_TEXT, "Compare Plans").get_attribute('href'), service_cards)
))

service_plan_data = []
for title, link in card_info:
    print(title, link)
    driver.get(link)
    wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.imh-rostrum-container"))
    )
    
    # There are different plans for different billing periods
    # So this code is to ensure the annual plan is selected
    period_btn = driver.find_element(
        By.XPATH, 
        f"//button[contains(@class, 'imh-term-selector') and text()='{period_string}']"
    )
    driver.execute_script("arguments[0].click();", period_btn)
    print(driver.find_element(By.CSS_SELECTOR, "button.imh-term-selector.active").text)
    
    # Make sure that data is updated
    wait.until(EC.text_to_be_present_in_element(
        (By.CSS_SELECTOR, "div.imh-term-selector > button.imh-term-selector.active"),
        period_string
    ))
    
    plans_container_all = BeautifulSoup(driver.page_source, 'html.parser')
    plans = plans_container_all.find('div', class_='imh-rostrum-container').find_all('div', class_='imh-rostrum-card')
    for plan in plans:
        title, description, discounted_price, normal_price, discount, features = get_plan_data(plan)
        service_plan_dict = {
            'service_title': title,
            'service_link': link,
            'period': period_string,
            'title': title,
            'description': description,
            'discounted_price': discounted_price,
            'normal_price': normal_price,
            'discount': discount,
            'features': features
        }
        
        service_plan_data.append(service_plan_dict)
        
    time.sleep(2)
    
pprint(service_plan_data)

main url https://www.inmotionhosting.com/
period 1 Year
Shared Hosting https://www.inmotionhosting.com/shared-hosting
1 Year
plan-title Core
plan-title Launch
plan-title Power
plan-title Pro
WordPress Hosting https://www.inmotionhosting.com/wordpress-hosting
1 Year
plan-title WP Core
plan-title WP Launch
plan-title WP Power
plan-title WP Pro
VPS Hosting https://www.inmotionhosting.com/vps-hosting
1 Year
plan-title VPS 4 vCPU
plan-title VPS 8 vCPU
plan-title VPS 12 vCPU
plan-title VPS 16 vCPU
Dedicated Hosting https://www.inmotionhosting.com/dedicated-servers
Standard
Affordable Dedicated Hosting
plan-title Aspire
plan-title Essential
plan-title Advanced
plan-title Elite
plan-title Extreme
[{'description': 'Suitable for simple sites, with everything you need '
                 'included.',
  'discount': '73%',
  'discounted_price': '$2.99',
  'features': ['Free domain & SSL',
               '2 Websites',
               '100GB SSD',
               'Unlimited Bandwidth',
               '2

In [283]:
import pandas as pd

df = pd.DataFrame(service_plan_data)
df.to_csv('service_plan_data.csv', index=False)

df.head()


,service_title,service_link,period,title,description,discounted_price,normal_price,discount,features
0,Core,https://www.inmotionhosting.com/shared-hosting,1 Year,Core,"Suitable for simple sites, with everything you...",$2.99,$10.99,73%,"[Free domain & SSL, 2 Websites, 100GB SSD, Unl..."
1,Launch,https://www.inmotionhosting.com/shared-hosting,1 Year,Launch,Designed for seamless multi-site management & ...,$4.79,$13.99,66%,"[Free domain & SSL, 25 Websites, Unlimited NVM..."
2,Power,https://www.inmotionhosting.com/shared-hosting,1 Year,Power,Powerful resources perfect for larger sites & ...,$4.79,$17.99,73%,"[Free domain & SSL, Unlimited Websites, Unlimi..."
3,Pro,https://www.inmotionhosting.com/shared-hosting,1 Year,Pro,Ideal for high-traffic or eCommerce sites with...,$10.79,$24.99,57%,"[Free domain & SSL, Unlimited Websites, Unlimi..."
4,WP Core,https://www.inmotionhosting.com/wordpress-hosting,1 Year,WP Core,,$3.49,$11.49,70%,"[Free domain & SSL, 2 Websites, 100GB NVMe SSD..."
